In [9]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline


In [10]:
sys.path.append("C:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble")

In [11]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/turns.csv', index_col='game_id')

In [12]:
import functions
import transformers
import importlib

importlib.reload(functions)
importlib.reload(transformers)
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_rows_transformer import SelectRowsTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer





from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_rows_transformer)
importlib.reload(transformers.series_from_group_transformer)




<module 'transformers.select_rows_transformer' from 'C:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\transformers\\select_rows_transformer.py'>

In [13]:
names = ['BetterBot', 'STEEBot', 'HastyBot']
bots = BotExtarctor(names)(train)
is_player = ~IsBotExtarctor(names)(train)


In [14]:
test = train
transformer = SelectTransformer(is_player)
transformer.transform(test)


,nickname,score,rating
game_id,,,
1,stevy,429,1500
3,davidavid,440,1811
4,Inandoutworker,119,1473
5,stevy,325,1500
6,HivinD,378,2029
...,...,...,...
72767,HAYDEN,340,1395
72770,samsiah06,97,1332
72771,BB-8,390,1500


In [23]:
pipe = (Pipeline([('bot_names', ExtractSetColumnsTransformer({'bot_name': BotExtarctor(names)})),
                   ('players_selection', SelectTransformer(is_player)),
                   ('set_bot_training', ColumnsSetterTransformer({'bot_rating': train[~is_player]['rating']})),
                   ('name_dropper', NameDropperTransformer(['nickname']))
                   ]))

new_train = pipe.transform(train)
new_train

,score,rating,bot_name,bot_rating
game_id,,,,
1,429,1500,BetterBot,1637
3,440,1811,BetterBot,2071
4,119,1473,BetterBot,1936
5,325,1500,STEEBot,1844
6,378,2029,STEEBot,2143
...,...,...,...,...
72767,340,1395,BetterBot,1952
72770,97,1332,HastyBot,2237
72771,390,1500,HastyBot,1614


In [24]:
new_games = games.merge(new_train, left_index=True, right_index=True)
new_games

,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,score,rating,bot_name,bot_rating
game_id,,,,,,,,,,,,,,,
1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,429,1500,BetterBot,1637
3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,440,1811,BetterBot,2071
4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,119,1473,BetterBot,1936
5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722,325,1500,STEEBot,1844
6,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541,378,2029,STEEBot,2143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,BetterBot,regular,TIME,0,2022-09-05 04:35:56,CSW21,3600,0,CASUAL,1,3669.988413,340,1395,BetterBot,1952
72770,HastyBot,regular,TIME,0,2022-09-04 00:13:24,CSW21,3600,0,RATED,1,3662.659287,97,1332,HastyBot,2237
72771,HastyBot,regular,STANDARD,0,2022-09-19 03:28:44,ECWL,900,0,CASUAL,10,236.133634,390,1500,HastyBot,1614


In [16]:
is_player = ~IsBotExtarctor(names)(train)
bots = BotExtarctor(names)(train)
train_no_bots = train[is_player]
train_no_bots['bot_name'] = bots
train_no_bots['bot_rating'] = train[~is_player]['rating']
train_no_bots = train_no_bots.drop(columns=['nickname'])
# games_with_rating = games.merge(train_no_bots, left_index=True, right_index=True)
train_no_bots


C:\Users\Bina4\AppData\Local\Temp\ipykernel_20688\2770609422.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_no_bots['bot_name'] = bots
C:\Users\Bina4\AppData\Local\Temp\ipykernel_20688\2770609422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_no_bots['bot_rating'] = train[~is_player]['rating']


,score,rating,bot_name,bot_rating
game_id,,,,
1,429,1500,BetterBot,1637
3,440,1811,BetterBot,2071
4,119,1473,BetterBot,1936
5,325,1500,STEEBot,1844
6,378,2029,STEEBot,2143
...,...,...,...,...
72767,340,1395,BetterBot,1952
72770,97,1332,HastyBot,2237
72771,390,1500,HastyBot,1614


In [17]:
turns

,turn_number,nickname,rack,location,move,points,score,turn_type
game_id,,,,,,,,
1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
1,4,stevy,EMORSUX,5K,UM,16,34,Play
1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...
72773,22,adola,ABINRRU,15N,IN,18,376,Play
72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
72773,24,adola,ABRRU,15E,BRA.,7,383,Play


In [18]:
trans = SelectRowsTransformer(train.index)
train_turns = trans.transform(turns)

In [25]:
trans_group = SeriesFromGroupTransformer('game_id', 'turn_number')
turns_seires = trans_group.transform(train_turns)

In [26]:
turns_seires[364]

,nickname,rack,location,move,points,score,turn_type
turn_number,,,,,,,
1,BB-8,DGINNPS,8D,PINGS,22,22,Play
2,HastyBot,CCEJRTY,D5,CRY.T,12,12,Play
3,BB-8,DEGNOTT,10C,TOT,16,38,Play
4,HastyBot,ACEEJNT,6D,.EJECT,31,43,Play
5,BB-8,DEEFGHN,7A,DEF.,15,53,Play
...,...,...,...,...,...,...,...
35,BB-8,U,NaN,-,0,312,Pass
36,HastyBot,U,NaN,-,0,361,Pass
37,BB-8,U,NaN,-,0,312,Pass
